# House Sales in king County, USA

- **COMPANY:** House Rocket.


- **BUSINESS MODEL:** Buy houses at a low price and review at the highest price.


- **WHAT'S THE CHALLENGE:** Finding good deals within the available portfolio, that is, finding homes with a low price, in a great location and that have a great resale potential for a higher price.


- **DATA:** This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.


- **SOURCE:** https://www.kaggle.com/harlfoxem/housesalesprediction

# Setup

## Imports

In [1]:
# Data Manipulation
import csv
import random
import datetime
import numpy               as np
import pandas              as pd

# Exploratory Data Analyses
import seaborn             as sns
import plotly.express      as px
import matplotlib.pyplot   as plt
#import matplotlib.gridspec as gridspec

#from tqdm import tqdm_notebook
#Configurations
import warnings
warnings.filterwarnings( 'ignore' )
#from IPython.display       import Image
#from IPython.core.display  import HTML

# graphic visualization
#import ipywidgets          as widgets
#from ipywidgets            import fixed

# data import
from geopy.geocoders       import Nominatim

#from scipy                 import stats     as ss
#from scipy.stats           import pointbiserialr, chi2_contingency

#import pickle





#from boruta                import BorutaPy
#from tabulate              import tabulate

## Helper Function

In [56]:
def descriptive_statistics(num_df):
    # Central Tendency - mean, median
    ct1 = pd.DataFrame(num_df.apply(np.mean)).T
    ct2 = pd.DataFrame(num_df.apply(np.median)).T

    # Dispersion - Std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(num_df.apply(np.std)).T
    d2 = pd.DataFrame(num_df.apply(min)).T
    d3 = pd.DataFrame(num_df.apply(max)).T
    d4 = pd.DataFrame(num_df.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_df.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_df.apply(lambda x: x.kurtosis())).T

    # Concatenate
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
    
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    
    return m

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
        

# Loading Data

In [3]:
df = pd.read_csv('/home/leandro/repos/House_Rocket_repo/data/raw/kc_house_data.csv')

In [18]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00,3,1.00,1180,5650,1.00,0,0,...,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,20141209T000000,538000.00,3,2.25,2570,7242,2.00,0,0,...,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,20150225T000000,180000.00,2,1.00,770,10000,1.00,0,0,...,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,20141209T000000,604000.00,4,3.00,1960,5000,1.00,0,0,...,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,20150218T000000,510000.00,3,2.00,1680,8080,1.00,0,0,...,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503


# Data Descripition

## Features Description

**id =**	Identification;

**date =**	Date sold;

**price =**	Sale price;

**bedrooms =**	Number of bedrooms;

**bathrooms =**	Number of bathrooms;

**sqft_liv =**	Size of living area in square feet;

**sqft_lot =**	Size of the lot in square feet;

**floors =**	Number of floors;

**waterfront =**	'1' if the property has a waterfront, '0' if not;

**view =**	An index from 0 to 4 of how good the view of the property was;

**condition =**	Condition of the house, ranked from 1 to 5. See session 2.1.1;

**grade =**	Classification by construction quality, ranked from 1 to 13. See session 2.1.2;

**sqft_above =**	Square feet above ground;

**sqft_basmt =**	Square feet below ground;

**yr_built =**	Year built;

**yr_renov =**	Year renovated. '0' if never renovated;

**zipcode =**	5 digit zip code;

**lat =**	Latitude;

**long =**	Longitude;

**squft_liv15 =**	Average size of interior housing living space for the closest 15 houses, in square feet;

**squft_lot15 =**	Average size of land lots for the closest 15 houses, in square feet;

**Shape_leng =**	Polygon length in meters;

**Shape_Area =**	Polygon area in meters;


 **Source:** https://geodacenter.github.io/data-and-lab//KingCounty-HouseSales2015/ 

### Relative to age and grade. Coded 1-5.

1 = Poor- Worn out. Repair and overhaul needed on painted surfaces, roofing, plumbing, heating and numerous functional inadequacies. Excessive deferred maintenance and abuse, limited value-in-use, approaching abandonment or major reconstruction; reuse or change in occupancy is imminent. Effective age is near the end of the scale regardless of the actual chronological age.

2 = Fair- Badly worn. Much repair needed. Many items need refinishing or overhauling, deferred maintenance obvious, inadequate building utility and systems all shortening the life expectancy and increasing the effective age.

3 = Average- Some evidence of deferred maintenance and normal obsolescence with age in that a few minor repairs are needed, along with some refinishing. All major components still functional and contributing toward an extended life expectancy. Effective age and utility is standard for like properties of its class and usage.

4 = Good- No obvious maintenance required but neither is everything new. Appearance and utility are above the standard and the overall effective age will be lower than the typical property.

5 = Very Good- All items well maintained, many having been overhauled and repaired as they have shown signs of wear, increasing the life expectancy and lowering the effective age with little deterioration or obsolescence evident with a high degree of utility.

Source: https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r


### Building Grade

Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:


**1-3 =** Falls short of minimum building standards. Normally cabin or inferior structure.


**4 =** Generally older, low quality construction. Does not meet code.


**5 =** Low construction costs and workmanship. Small, simple design.


**6 =** Lowest grade currently meeting building code. Low quality materials and simple designs.


**7 =** Average grade of construction and design. Commonly seen in plats and older sub-divisions.


**8 =** Just above average in construction and design. Usually better materials in both the exterior and interior finish work.


**9 =** Better architectural design with extra interior and exterior design and quality.


**10 =** Homes of this quality generally have high quality features. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.


**11 =** Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.


**12 =** Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.


**13 =** Generally custom designed and built. Mansion level. Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.


**Source:** https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r

### Bathrooms

0.25 = A 0.25 bathroom is a bathroom that has either a sink, a shower, toilet or a bathtub.

0.50 = A 0.5 bathroom is a bathroom that does not contain a bath or a shower, just a toilet and sink.

0.75 = A 0.75 bathroom is a bathroom that has either a shower or tub with 1 sink and 1 toilet.

1.00 = A full bathroom (1.0) is a bathroom that contains exactly 1 sink, 1 bathtub, 1 shower and 1 toilet.

Source: https://www.badeloftusa.com/buying-guides/bathrooms/


## Data Dimentions

In [31]:
print('Number of rows : {}'.format(data.shape[0]),end = '\n\n')
print('Number of cols : {}'.format(data.shape[1]),end = '\n\n' )

Number of rows : 21613

Number of cols : 21



In [32]:
## I need explicar floors

In [33]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [48]:
data['floors'].value_counts()

1.00    10680
2.00     8241
1.50     1910
3.00      613
2.50      161
3.50        8
Name: floors, dtype: int64

## Data Types

In [49]:
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

## Check NA

In [50]:
data.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## Change Types

In [52]:
data['date'] = pd.to_datetime(data['date'])

In [53]:
data.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

## Descriptive Statistics

In [54]:
num_attributes = df.select_dtypes(include = ['int64', 'int32', 'float64'])
cat_attributes = df.select_dtypes(exclude = ['int64', 'int32', 'float64'])

### Numerical Attributes

In [57]:
descriptive_statistics(num_attributes)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,id,1000102.00,9900000190.00,9899000088.00,4580301520.86,3904930410.00,2876499023.43,0.24,-1.26
1,price,75000.00,7700000.00,7625000.00,540088.14,450000.00,367118.70,4.02,34.59
2,bedrooms,0.00,33.00,33.00,3.37,3.00,0.93,1.97,49.06
3,bathrooms,0.00,8.00,8.00,2.11,2.25,0.77,0.51,1.28
4,sqft_living,290.00,13540.00,13250.00,2079.90,1910.00,918.42,1.47,5.24
5,sqft_lot,520.00,1651359.00,1650839.00,15106.97,7618.00,41419.55,13.06,285.08
6,floors,1.00,3.50,2.50,1.49,1.50,0.54,0.62,-0.48
7,waterfront,0.00,1.00,1.00,0.01,0.00,0.09,11.39,127.63
8,view,0.00,4.00,4.00,0.23,0.00,0.77,3.40,10.89
9,condition,1.00,5.00,4.00,3.41,3.00,0.65,1.03,0.53
